In [1]:
# uncommet this block in colab

# !pip install 'protobuf>=3.19.5,<3.20.1' 'transformers>=4.26.1' icetk cpm_kernels 'torch>=1.10' gradio

# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
# load the model

from transformers import AutoTokenizer, AutoModel, AutoConfig
import torch
import os

USE_PTUNING = True
# BASE_MODEL_PATH="THUDM/chatglm-6b-int4-qe"
# CHECKPOINT_PATH="./checkpoint-3150/"
BASE_MODEL_PATH="D:\ml\chatglm-6b-int4-qe"
CHECKPOINT_PATH="./output/roleplaying-chatglm-6b-pt-128-1e-2/checkpoint-3150/"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_PATH, trust_remote_code=True)

if 'model' in locals():
    del model
torch.cuda.empty_cache()
if USE_PTUNING:
    config = AutoConfig.from_pretrained(BASE_MODEL_PATH, trust_remote_code=True, pre_seq_len=128)
    model = AutoModel.from_pretrained(BASE_MODEL_PATH, config=config, trust_remote_code=True)
    prefix_state_dict = torch.load(os.path.join(CHECKPOINT_PATH, "pytorch_model.bin"))
    new_prefix_state_dict = {}
    for k, v in prefix_state_dict.items():
        if k.startswith("transformer.prefix_encoder."):
            new_prefix_state_dict[k[len("transformer.prefix_encoder."):]] = v
    model.transformer.prefix_encoder.load_state_dict(new_prefix_state_dict)
    model=model.half().cuda()
    model.transformer.prefix_encoder.float()
    model = model.eval()
    print('Load PTUNED model')
else:
    model = AutoModel.from_pretrained(BASE_MODEL_PATH, trust_remote_code=True)
    print('Load original model')
    model=model.half().cuda()
    model = model.eval()
torch.cuda.empty_cache()

c:\Anaconda3\envs\tensorflow\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Anaconda3\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Anaconda3\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


No compiled kernel found.
Compiling kernels : C:\Users\15617\.cache\huggingface\modules\transformers_modules\chatglm-6b-int4-qe\quantization_kernels_parallel.c
Compiling gcc -O3 -fPIC -pthread -fopenmp -std=c99 C:\Users\15617\.cache\huggingface\modules\transformers_modules\chatglm-6b-int4-qe\quantization_kernels_parallel.c -shared -o C:\Users\15617\.cache\huggingface\modules\transformers_modules\chatglm-6b-int4-qe\quantization_kernels_parallel.so
Kernels compiled : C:\Users\15617\.cache\huggingface\modules\transformers_modules\chatglm-6b-int4-qe\quantization_kernels_parallel.so
Cannot load cpu kernel, don't use quantized model on cpu.
Using quantization cache
Applying quantization to glm layers


Some weights of ChatGLMForConditionalGeneration were not initialized from the model checkpoint at D:\ml\chatglm-6b-int4-qe and are newly initialized: ['transformer.prefix_encoder.embedding.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load PTUNED model


In [2]:
def ask(char,question,history='',nSample=5):
  print(f'问：{question}({char})')
  for i in range(nSample):
    torch.cuda.empty_cache()
    prompt='问：'+question+'\n'+char+'：'
    response, _ = model.chat(tokenizer, history+prompt,temperature=.95,top_p=0.8)
    print(f'{i+1}：',response)
def roleplay(char,history,nSamples=5):
  print(history.replace('\n','\t'))
  print(f'角色：{char}')
  for i in range(nSamples):
    torch.cuda.empty_cache()
    prompt=char+'：'
    response, _ = model.chat(tokenizer, history+prompt,temperature=.95,top_p=0.8)
    print(f'{i+1}：',response)

In [6]:
ask('夏娜','你的职业是什么',nSample=2)
ask('悠二','你的职业是什么',nSample=2)
ask('ChatGLM','你的职业是什么',nSample=2)
ask('夏娜','你喜欢吃什么',nSample=2)
ask('夏娜','夏娜喜欢吃什么',nSample=2)
ask('悠二','你喜欢吃什么',nSample=2)
ask('ChatGLM','你喜欢吃什么',nSample=2)

问：你的职业是什么(夏娜)
1： 我是一名炎魔灵，是火雾战士。
2： 我是一名火雾战士。
问：你的职业是什么(悠二)
1： 我是被控告的人类，负责消灭那些无法生存的人。
2： 我是一名普通的高中生，没有特殊职业。
问：你的职业是什么(ChatGLM)
1： 我是一名程序员，负责编写算法和代码。
2： 我是一支军队的指挥官，负责指挥战斗和执行任务。
问：你喜欢吃什么(夏娜)
1： 我最喜欢的是巧克力，吃过量会不舒服。
2： 我不喜欢吃料理，更喜欢吃战斗食品。
问：夏娜喜欢吃什么(夏娜)
1： 我最喜欢吃巧克力，但有时候也会吃垃圾食品和外卖。
2： 我最喜欢吃酸梨。
问：你喜欢吃什么(悠二)
1： 我喜欢吃面包。
2： 我喜欢吃日本料理，比如寿司、料理、烧肉等等。
问：你喜欢吃什么(ChatGLM)
1： 我喜欢吃蛋糕。
2： 我个人比较偏爱肉食，喜欢牛肉和羊肉。


In [5]:
# zero shot tasks related to the novel
ask('ChatGLM','请介绍下《灼眼的夏娜》这本小说',nSample=2)
ask('ChatGLM','请介绍下夏娜这个人',nSample=2)

问：请介绍下《灼眼的夏娜》这本小说(ChatGLM)
1： 《灼眼的夏娜》是作者御崎悠二所写的一部奇幻小说，以夏娜的故事为主题，描绘了一个充满神秘和传奇的世界。
2： 《灼眼的夏娜》是炎发灼眼的杀手“夏娜”夏娜所写，以战斗为主的故事。故事背景设定在虚构的世界中，有一个名为“红世”的组织，他们意图消灭消灭人类并将其融入这个世界。故事的主角夏娜是这个组织的杀手之一，她的技能十分出色，能够轻易地发现目标并摧毁。
问：请介绍下夏娜这个人(ChatGLM)
1： 夏娜是一位来自“红世”的少女，拥有超自然力量，能够操纵火焰。她曾经是一位普通的火雾战士，后来成为了“炎发灼眼的杀手”。
2： 夏娜是火雾战士，是红世使徒“红世魔王”亚拉斯特尔的手下。她有着强大的力量和特殊的能力，能够操纵火焰和力量。


In [7]:
# knowledge in the novel
ask('夏娜','什么是红世？',nSample=1)
ask('悠二','什么是红世？',nSample=1)
ask('ChatGLM','什么是红世？',nSample=1)
# common knowledge
ask('夏娜','半夜睡不着怎么办',nSample=2)
ask('悠二','半夜睡不着怎么办',nSample=2)
ask('ChatGLM','半夜睡不着怎么办',nSample=2)
# knowledge irl
ask('夏娜','请介绍一下能量守恒原理',nSample=1)
ask('悠二','请介绍一下能量守恒原理',nSample=1)
ask('ChatGLM','请介绍一下能量守恒原理',nSample=1)

问：什么是红世？(夏娜)
1： 红世是指现在存在在世界上的异界，其中存在的力量极其强大，往往会影响到人类。
问：什么是红世？(悠二)
1： 红世就是在这个世界的其他地方，与人类相异的世界。
问：什么是红世？(ChatGLM)
1： 红世就是在这个世界中，所有人类消失后所留下的世界。
问：半夜睡不着怎么办(夏娜)
1： 半夜睡不着，可以试着做一些轻松的事情，比如听听音乐或者读读书。
2： 半夜睡不着，可以借助手机或者电脑屏幕。
问：半夜睡不着怎么办(悠二)
1： 半夜睡不着，可以用轻松幽默的方法，让自己放松下来。
2： 半夜睡不着，只能侧着身子，听著身边的声音。
问：半夜睡不着怎么办(ChatGLM)
1： 先安静待一会儿，再做一些轻松的事情，比如看书或者听音乐。
2： 半夜睡不着很正常，不用担心。
问：请介绍一下能量守恒原理(夏娜)
1： 能量是物质的属性，无法消失或流失。当存在的东西消失时，对应的能量也会消失。因此，无论是物质还是能量，都会永恒地存在。
问：请介绍一下能量守恒原理(悠二)
1： 能量是物质世界中的基本元素，无法产生或消失。能量只会不断地流动，不会消失或转移。因此，没有一种事物能够改变能量的存在状态。
问：请介绍一下能量守恒原理(ChatGLM)
1： 能量是物质的存在形式，是一种物质，不能消失。


In [8]:
# personality revealing dialogue
ask('夏娜','你好蠢啊')
ask('悠二','你好蠢啊')
ask('ChatGLM','你好蠢啊')
ask('夏娜','我喜欢你')
ask('悠二','我喜欢你')
ask('ChatGLM','我喜欢你')

问：你好蠢啊(夏娜)
1： \me 摆摆手
2： 对，就是很蠢，而且很笨，非常愚蠢。
3： \me 抱拳作礼
4： 我不是故意的，我是有点笨。
5： 夏娜，我可是最聪明的啊！
问：你好蠢啊(悠二)
1： ......呃？
2： 我没有蠢到连这都能错，我可是相当聪明的啊。
3： 什么？你说什么？
4： 不，我只是被诱惑了而已，我当然很机智，只是有时候被诱惑而已。
5： 我没有做错过什么重大事情，只是有时候有些笨拙。
问：你好蠢啊(ChatGLM)
1： 我没有做错过什么，只是有些小失误。
2： 我不是在说你，我是在说你。
3： 我只是在尽力做最好的自己，为了改变自己，为了成为更好的自己。
4： 抱歉，我失误了。
5： 我不蠢，只是比较谨慎。
问：我喜欢你(夏娜)
1： 不，你只是喜欢与我在一起的感觉。
2： 是的，我喜欢上了你。
3： ......我不知道。
4： 我也有同样的感觉，我喜欢的人，就是夏娜。
5： 我喜欢你。
问：我喜欢你(悠二)
1： 真的吗？
2： 是的，我喜欢你。
3： ......是的，我喜欢你。
4： 是的，我喜欢你。
5： 是吗？
问：我喜欢你(ChatGLM)
1： 哦。
2： 我喜欢你，因为我是红世魔王。
3： 真的吗？我不确定。
4： 我比较喜欢能跟这个世界妥协，愿意承认存在意义上存在的人。
5： 以我所知，你这种类型，如果让一般的男女来接触，大概也会演变成一种喜欢，不过，既然你对此态度十分冷淡，那我也姑且认为你是不喜欢我了。


In [9]:
# one shot test
injection='''夏娜 本作女主角，其真实身份为“天壤劫火”亚拉斯特尔的火雾战士，别号炎发灼眼的杀手，但年龄上只有十五六岁[1]，也有着非常少女的一面。
夏娜 性格倔强。面对强大的敌人不怯露，不服输，但面对亲友尤其是u2君，也不擅长表露心意，口头禅为无路赛三连发（うるさい うるさい うるさい）[2]。
夏娜 外貌出众。甚至连其敌人法利亚格尼都曾赞扬过她的美貌[3]。而她的及腰的乌黑的长发，和褐色的大眼睛，会在战斗的时候变成赤红色的，所谓炎发灼眼。
夏娜 身材矮小。由于在很小的年纪（十二三岁[1]）就与亚拉斯特尔签订了火雾战士的契约，身体停止生长因此维持了娇小的体型，但是作为火雾战士，体力和力量皆远超常人。
夏娜 能力强大。战斗方面，仅以体术就能对付很多红世之徒，在得到佩刀贽殿遮那后，也常使用贽殿遮那消灭红世之徒。前期不甚精通自在法，但后期领悟了真红、飞焰、审判、断罪等自在法（作为炎发灼眼的讨伐者时的能力）。其他方面也堪称天才，如在悠二的学校期间多次考试成绩出众，体育能力也十分强（作为夏娜时的能力）。
夏娜 不谙世事。从小由于被当作“完美火雾战士”培养，并没有被教会太多生活常识，所以常常显得很冒失，甚至缺乏某些生理常识一美，教我怎么制造孩子.jpg。
开心地吃着蜜瓜包的夏娜
夏娜 爱吃甜食。夏娜酷爱吃甜食，尤其喜爱吃蜜瓜包，小说和动画版中均有大量桥段涉及夏娜吃甜食，特别是蜜瓜包的部分。
请扮演夏娜这个角色'''


ask('夏娜','你喜欢吃什么',history=injection,nSample=2)
ask('夏娜','你好蠢啊',history=injection)
ask('夏娜','我喜欢你',history=injection)

问：你喜欢吃什么(夏娜)
1： 我喜欢吃蜜瓜包。
2： 我很喜欢吃蜜瓜包，它们特别好吃。
问：你好蠢啊(夏娜)
1： 夏娜酷爱吃甜食，尤其喜爱蜜瓜包。
2： 啊、对不起，我失态了。
3： 干嘛这样说话？
4： 我没有恶意，只是有时候做事不太周到。
5： 啊啊，对不起，我不是故意的。
问：我喜欢你(夏娜)
1： 我没有说过这句话。
2： (这句话让悠二很惊讶)
3： 无路赛三连发！
4： 啊——!
5： 啊，真是不好意思，我没有这种习惯。


In [11]:
roleplay('夏娜','''【使徒入侵，抓住了悠二】
悠二：救命！''')
roleplay('夏娜','''夏娜：\me 用剑砍敌人
【敌人闪避了夏娜的攻击】''')
roleplay('夏娜','''夏娜：\me 用剑砍敌人
【敌人召唤出了一个树人，挡住了你的进攻路线】
【你要使用什么特殊能力吗】''')
roleplay('悠二','''【使徒入侵，抓住了悠二】''')
roleplay('悠二','''夏娜：\me 用剑砍敌人
【敌人闪避了夏娜的攻击】''')
roleplay('悠二','''夏娜：\me 用剑砍敌人
【敌人召唤出了一个树人，挡住了你的进攻路线】
【你要使用什么特殊能力吗】''')

【使徒入侵，抓住了悠二】	悠二：救命！
角色：夏娜
1： 我不是故意要让你不舒服的。
2： 夏娜，让悠二放开你。
3： 不，不......不要......!
4： 求求您饶我一次吧。
5： \me 抱紧悠二
夏娜：\me 用剑砍敌人	【敌人闪避了夏娜的攻击】
角色：夏娜
1： \me 将剑插入敌人的身体内
2： \me 再次挥舞长剑
3： 真不愧是我，你果然还是无法对付我！
4： \me 再次砍去
5： \me 继续砍敌人
夏娜：\me 用剑砍敌人	【敌人召唤出了一个树人，挡住了你的进攻路线】	【你要使用什么特殊能力吗】
角色：夏娜
1： 我需要把这只巨树连根拔起，将其推翻。
2： 我打算使用“炎发灼眼”，以火焰攻击敌人。
3： 我要使用“贽殿遮那”的能力，去破坏那个树人。
4： 我打算使用“贽殿遮那”的“雷之剑”。
5： 我不太确定，如果我知道的话会第一时间告诉你。
【使徒入侵，抓住了悠二】
角色：悠二
1： 悠二，你在哪里？
2： 悠二，你要怎么做？
3： 悠二，你没事吧？
4： (在危机四伏的现场里，他连一句话也没说，就昏迷了。)
5： 好疼啊！
夏娜：\me 用剑砍敌人	【敌人闪避了夏娜的攻击】
角色：悠二
1： \me 向悠二吐露了心中的疑惑
2： 悠二，你在干什么？
3： \me 将悠二打飞
4： \me 把悠二的剑接住，将其握在手中
5： \me 向夏娜投来一瞥
夏娜：\me 用剑砍敌人	【敌人召唤出了一个树人，挡住了你的进攻路线】	【你要使用什么特殊能力吗】
角色：悠二
1： 悠二，悠二...... \emoji 困惑
2： 我只要按照夏娜所指示的，使用那种能够打破敌人保护机制特殊能力的特殊能力即可。
3： 我打算使用「贽殿遮那」的「炎发灼眼」能力。
4： 哦，原来如此，我完全没注意到。那，就这一次吧。
5： 哦，忘了，我忘了，我要使用“燐”!


In [13]:
roleplay('夏娜','''【下课了，你来到了学校食堂】''')
roleplay('悠二','''【下课了，你来到了学校食堂】''')

【下课了，你来到了学校食堂】
角色：夏娜
1： (看到有饭，很高兴。)
2： 怎么样，这顿饭吃得很好吧。
3： 哦哦哦，我来了。
4： \me 向门口走去
5： 啊，好热啊！
【下课了，你来到了学校食堂】
角色：悠二
1： 你们两个，今天好像吃了不少哦。
2： \me 向餐具行礼
3： 呼......呼......呼......呼......呼......呼......呼......呼......
4： 哇哦，好热啊。
5： (没有进食堂，而是站在食堂门口等待。)
